<a href="https://colab.research.google.com/github/md-marop-hossain/Ocular-Disease-Recognition-Project-CSE499A/blob/main/Ocular_Disease_Recognition_using_CNN_models_resnet_50_%26_VGG_16_%26_Dense_Net_121.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
from google.colab import files
files.upload()
#create a kaggle folder
! mkdir ~/.kaggle
#copy the kaggle .josn to folder created
! cp kaggle.json ~/.kaggle/
#permission for the json the act
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

Saving kaggle.json to kaggle.json
ref                                                                   title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
nelgiriyewithana/countries-of-the-world-2023                          Global Country Information Dataset 2023             23KB  2023-07-08 20:37:33          11179        445  1.0              
nelgiriyewithana/global-youtube-statistics-2023                       Global YouTube Statistics 2023                      60KB  2023-07-28 15:36:38           3313        130  1.0              
joebeachcapital/global-earth-temperatures                             Global Earth Temperatures                           33KB  2023-08-09 06:21:51            701         29  1.0              
j

In [2]:
! kaggle datasets download -d andrewmvd/ocular-disease-recognition-odir5k

100% 1.62G/1.62G [00:21<00:00, 78.8MB/s]
100% 1.62G/1.62G [00:21<00:00, 81.4MB/s]


**Datasets**

Ocular Disease Intelligent Recognition (ODIR) is a structured
ophthalmic database of 5,000 patients with age, color fundus
photographs from left and right eyes and doctors' diagnostic
keywords from doctors.

This dataset is meant to represent ‘‘real-life’’ set of patient
information collected by Shanggong Medical Technology Co.,
Ltd. from different hospitals/medical centers in China. In these
institutions, fundus images are captured by various cameras in
the market, such as Canon, Zeiss and Kowa, resulting into
varied image resolutions.

Annotations were labeled by trained human readers with quality
control management. They classify patient into eight labels
including:

- - Normal (N),
- - Diabetes (D),
- - Glaucoma (G),
- - Cataract (C),
- - Age related Macular Degeneration (A),
- - Hypertension (H),
- - Pathological Myopia (M),
- - Other diseases/abnormalities (O)

In [ ]:
! unzip ocular-disease-recognition-odir5k

In [4]:
import numpy as np
import pandas as pd
import cv2 #It's a library that provides functions for computer vision tasks, such as image processing, feature extraction, and object detection.
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

**`numpy (import numpy as np):`**

 - - **`numpy`** is a widely used library for numerical computations in Python.

- - The np alias is commonly used to refer to numpy functions and classes.

**`pandas (import pandas as pd):`**

- - **`pandas`** is a data manipulation library that provides data structures like DataFrames, which are used to store and manipulate tabular data.

**`cv2:`**

- - cv2 stands for OpenCV (Open Source Computer Vision Library).
- - It's a library that provides functions for computer vision tasks, such as image processing, feature extraction, and object detection.

**`random:`**

- - The random module is used to generate random numbers and perform random operations.

**`tqdm:`**

- - tqdm is a library that provides a progress bar for loops, making it easier to track the progress of lengthy computations.

**`matplotlib.pyplot (import matplotlib.pyplot as plt):`**

- - matplotlib is a popular plotting library for creating visualizations and graphs.
- - The pyplot submodule provides an interface similar to MATLAB's plotting.

**`sklearn.metrics:`**

- - sklearn (scikit-learn) is a machine learning library in Python.
metrics submodule contains functions for evaluating model performance, calculating metrics like accuracy, precision, recall, etc.

**`sklearn.model_selection:`**

- - This submodule contains tools for model selection and evaluation, such as splitting data into training and testing sets.

**`sklearn.preprocessing:`**

- - preprocessing submodule contains utilities for data preprocessing tasks, like scaling, transforming, and encoding data.

**`tensorflow.keras.preprocessing.image:`**

- - tensorflow.keras is part of the TensorFlow library and provides tools for building and training deep learning models.

- - The preprocessing.image submodule contains functions for loading and preprocessing images.

**`tensorflow.keras.applications.ResNet50:`**

- - ResNet50 is a popular deep neural network architecture for image classification.
- - It's pre-trained on a large dataset and can be used for transfer learning.

**`tensorflow.keras.models.Sequential:`**

- - Sequential is a class in Keras that allows you to create a linear stack of layers for building neural network models.

**`tensorflow.keras.layers:`**

- - This module provides various types of layers that can be added to a neural network model, such as dense (fully connected), convolutional, recurrent, etc.

**`tensorflow.keras.optimizers.Adam:`**

- - Adam is an optimization algorithm used to update the weights of a neural network during training.

**`tensorflow.keras.callbacks.EarlyStopping:`**

- - EarlyStopping is a callback that stops training when a monitored metric (like validation loss) stops improving, preventing overfitting.

In [5]:
#set random seed for reproducibility
random.seed(42)

In [6]:
#read the CSV file
df = pd.read_csv('/content/full_df.csv')
df.head(5)

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg
